####In this document we will see how to calculate the FLOPs for KERAS model

In [ ]:
#Install package
#Requirements Python 3.6+ & Tensorflow 2.2+

!pip install keras-flops

In [2]:
#import packages

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose,  BatchNormalization, Activation


In [18]:
inp = Input((512, 512, 1))
x = BatchNormalization(input_shape=(512, 512, 1),name='input0',)(inp)

x = Conv2D(8, (3, 3), input_shape=(512, 512, 1), padding='same',strides=(1, 1), activation='relu')(x)
x = Activation('relu')(x)

x = Conv2D(16, (3, 3), input_shape=(512, 512, 1), padding='same',strides=(1, 1), activation='relu')(x)
x = Activation('relu')(x)

x = Conv2DTranspose(32, (3, 3), input_shape=(512, 512, 1), padding='same',strides=(1, 1), activation='relu')(x)
x = Activation('relu')(x)

x = Conv2DTranspose(16, (5, 5), input_shape=(512, 512, 1), padding='same',strides=(1, 1), activation='relu')(x)
x = Activation('relu')(x)

out = Conv2DTranspose(1, (5, 5), input_shape=(512, 512, 1), padding='same',strides=(1, 1), activation='relu',name='output0')(x)
model = Model(inp, out)

In [19]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 512, 512, 1)]     0         
_________________________________________________________________
input0 (BatchNormalization)  (None, 512, 512, 1)       4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 512, 512, 8)       80        
_________________________________________________________________
activation_12 (Activation)   (None, 512, 512, 8)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 512, 512, 16)      1168      
_________________________________________________________________
activation_13 (Activation)   (None, 512, 512, 16)      0         
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 512, 512, 32)      4640

####Note : 1 MACC = 2 Flops (1 Mutlitiplication + 1 Addition)

In [21]:
from keras_flops import get_flops
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {flops / 10 ** 9:.7} GFlops or BFlops")
print(f"MACCs: {0.5*flops / 10 ** 9:.7} MACCs")

FLOPs: 9.99791 GFlops or BFlops
MACCs: 4.998955 MACCs


#Tried and not working properly

##### All the below get_flops function was not working fo the custom developed model shown above




In [22]:
def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()


    with graph.as_default():
        with session.as_default():
            model = tf.keras.models.load_model(model_h5_path)

            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

            # Optional: save printed results to file
            # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
            # opts['output'] = 'file:outfile={}'.format(flops_log_path)
            # We use the Keras session graph in the call to the profiler.
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)

            return flops.total_float_ops

In [23]:
def get_flops2():
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()

    with graph.as_default():
        with session.as_default():
            #model = tf.keras.applications.MobileNet(
             #       alpha=1, weights=None, input_tensor=tf.compat.v1.placeholder('float32', shape=(1, 224, 224, 3)))
            model = model1
            run_meta = tf.compat.v1.RunMetadata()
            opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

            # Optional: save printed results to file
            # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
            # opts['output'] = 'file:outfile={}'.format(flops_log_path)

            # We use the Keras session graph in the call to the profiler.
            flops = tf.compat.v1.profiler.profile(graph=graph,
                                                  run_meta=run_meta, cmd='op', options=opts)

    tf.compat.v1.reset_default_graph()

    return flops.total_float_ops

In [ ]:
# This code is for older version of tensorflow
import tensorflow as tf
import keras.backend as K


def get_flops_0():
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=K.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops  # Prints the "flops" of the model.


# .... Define your model here ....
# You need to have compiled your model before calling this.
print(get_flops_0())